In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import joblib
# import hvplot.pandas

### Load and select data

In [3]:
data_preprocessed = pd.read_csv(r'C:\Users\cmcampos.REDESESCRIO\Desktop\Filmes Tera\projeto_filme_tera\bases_de_dados\bases iniciais\df_filmes_tratada.csv')
pd.set_option('display.max_columns', 150)
data_preprocessed.head()

,Film,Rotten Tomatoes % critics,Metacritic % critics,Average critics %,Rotten Tomatoes Audience %,Metacritic Audience %,Rotten Tomatoes vs Metacritic % deviance,Average audience %,Audience vs Critics deviance %,Primary Genre,Genres,Script Type,Opening weekend ($million),Opening Weekend,Domestic gross ($million),Domestic Gross,Foreign Gross ($million),Foreign Gross,Worldwide Gross,Worldwide Gross ($million),% of Gross earned abroad,Budget ($million),% Budget recovered,% Budget recovered opening weekend,Oscar Winners,Oscar Detail,Link,Rank,Title,Genre,Description,Director,Actors,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Success,sucesso_recalculado,ano
0,Lion,87.0,68.0,77.5,92.0,72.0,20.0,82.0,-4.5,drama,drama,adaptation,0.123360,123360.0,7.187095,7187095.0,3.150000,3150000,1.033710e+07,10.337095,0.304728,0.00,0.000000,0.000000,0,0,https://www.amazon.com/Lion-Dev-Patel/dp/B01NH...,19.0,Lion,"biography,drama",A five-year-old Indian boy gets lost on the st...,Garth Davis,"Dev Patel, Nicole Kidman, Rooney Mara, Sunny P...",118.0,8.1,102061.0,51.69,69.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2016.0
1,No Escape,46.0,38.0,42.0,67.0,65.0,2.0,66.0,-24.0,action,action,original screenplay,8.111264,8111264.0,27.203371,27203371.0,27.130000,27130000,5.433337e+07,54.333371,0.499325,0.45,120.740824,18.025031,0,0,http://www.amazon.com/gp/product/B016Z411SE/?t...,924.0,No Escape,"action,thriller","In their new overseas home, an American family...",John Erick Dowdle,"Lake Bell, Pierce Brosnan, Owen Wilson,Chatcha...",103.0,6.8,57921.0,27.29,38.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,2015.0
2,Insidious,67.0,52.0,59.5,65.0,70.0,-5.0,67.5,-8.0,horror,"horror, thriller",original screenplay,13.270000,13270000.0,54.010000,54010000.0,43.000000,43000000,6.728005e+07,67.280054,0.639120,1.50,44.853369,8.846667,0,0,http://www.amazon.com/gp/product/B016887PIW/?t...,653.0,Insidious,"horror,mystery,thriller",A family looks to prevent evil spirits from tr...,James Wan,"Patrick Wilson, Rose Byrne, Ty Simpkins, Lin S...",103.0,6.8,219916.0,53.99,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,2011.0
3,Annabelle,29.0,37.0,33.0,38.0,53.0,-15.0,45.5,-12.5,horror,horror,sequel,37.134255,37134255.0,84.273813,84273813.0,171.000000,171000000,2.552738e+08,255.273813,0.669869,6.50,39.272894,5.712962,0,0,http://www.amazon.com/gp/product/B018C4RSIM/?t...,819.0,Annabelle,"horror,mystery,thriller",A couple begins to experience terrifying super...,John R. Leonetti,"Ward Horton, Annabelle Wallis, Alfre Woodard,T...",99.0,5.4,91106.0,84.26,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,2014.0
4,Split,74.0,62.0,68.0,79.0,69.0,10.0,74.0,-6.0,horror,"horror, thriller",original screenplay,40.010975,40010975.0,138.141585,138141585.0,140.162993,140162993,2.783046e+08,278.304578,0.503632,9.00,30.922731,4.445664,0,0,https://www.amazon.co.uk/Split-James-McAvoy/dp...,3.0,Split,"horror,thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",117.0,7.3,157606.0,138.12,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,2017.0


In [4]:
data_preprocessed.columns

Index(['Film', 'Rotten Tomatoes % critics', 'Metacritic % critics',
       'Average critics %', 'Rotten Tomatoes Audience %',
       'Metacritic Audience %', 'Rotten Tomatoes vs Metacritic % deviance',
       'Average audience %', 'Audience vs Critics deviance %', 'Primary Genre',
       'Genres', 'Script Type', 'Opening weekend ($million)',
       'Opening Weekend', 'Domestic gross ($million)', 'Domestic Gross',
       'Foreign Gross ($million)', 'Foreign Gross', 'Worldwide Gross',
       'Worldwide Gross ($million)', '% of Gross earned abroad',
       'Budget ($million)', '% Budget recovered',
       '% Budget recovered opening weekend', 'Oscar Winners', 'Oscar Detail',
       'Link', 'Rank', 'Title', 'Genre', 'Description', 'Director', 'Actors',
       'Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)',
       'Metascore', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music',
       'Musical',

retiramos algumas colunas que nao fazem sentido pro modelo, como ano; e as variaveis que usamos para construir a variavel sucesso (media de critica e % do budget recuperado)

In [4]:
data_preprocessed.Director.value_counts()

Ridley Scott          7
Antoine Fuqua         5
Peter Berg            5
Paul W.S. Anderson    5
Michael Bay           5
                     ..
John Francis Daley    1
Kathryn Bigelow       1
Babak Najafi          1
Mark Herman           1
Mike Mills            1
Name: Director, Length: 348, dtype: int64

In [5]:
data_preprocessed = data_preprocessed[['Script Type', 'Budget ($million)', 'Opening weekend ($million)',
       'Oscar Winners', 'Director',
       'Runtime (Minutes)', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War',
       'Western']]

### Feature engineering

In [6]:
data_with_target = data_preprocessed.copy()
data_with_target.head()

,Script Type,Budget ($million),Opening weekend ($million),Oscar Winners,Director,Runtime (Minutes),Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,adaptation,0.00,0.123360,0,Garth Davis,118.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,original screenplay,0.45,8.111264,0,John Erick Dowdle,103.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,original screenplay,1.50,13.270000,0,James Wan,103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,sequel,6.50,37.134255,0,John R. Leonetti,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,original screenplay,9.00,40.010975,0,M. Night Shyamalan,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
# Obter as colunas categóricas
colunas_categoricas = ['Script Type', 'Director']

# Suponha que 'df' seja o seu DataFrame e 'categoria' seja a coluna categórica
encoder = OneHotEncoder(sparse=False)  # Use sparse=False para obter um array NumPy em vez de uma matriz esparsa

# Ajuste o codificador aos dados
categorias_encoded = encoder.fit_transform(data_with_target[colunas_categoricas])

# Crie um novo DataFrame com as colunas one-hot codificadas
categorias_encoded_df = pd.DataFrame(categorias_encoded, columns=encoder.get_feature_names_out(colunas_categoricas))

# Concatene 'categorias_encoded_df' ao DataFrame original para incluir as colunas one-hot
data_with_target = pd.concat([data_with_target, categorias_encoded_df], axis=1)

# Remova a coluna original 'categoria' se desejar
data_with_target.drop(['Script Type', 'Director'], axis=1, inplace=True)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [11]:
data_with_target

,Budget ($million),Opening weekend ($million),Oscar Winners,Runtime (Minutes),Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Script Type_adaptation,"Script Type_adaptation, based on a true story","Script Type_adaptation, remake / reboot",Script Type_based on a true story,Script Type_original screenplay,Script Type_remake / reboot,Script Type_sequel,"Script Type_sequel, adaptation",Director_Adam McKay,Director_Adam Shankman,Director_Adam Wingard,Director_Akiva Schaffer,Director_Alan Taylor,Director_Alejandro González Iñárritu,Director_Alessandro Carloni,Director_Alex Garland,Director_Alex Proyas,Director_Alexander Payne,Director_Alfonso Cuarón,Director_Alfonso Gomez-Rejon,Director_Allen Coulter,Director_Amma Asante,Director_Andrew Niccol,Director_Andrew Stanton,Director_Andrés Muschietti,Director_Andy Tennant,Director_Ang Lee,Director_Angelina Jolie,Director_Anna Foerster,Director_Anthony Russo,Director_Antoine Fuqua,Director_Ari Sandel,Director_Ariel Vromen,Director_Asger Leth,Director_Ash Brannon,Director_Babak Najafi,Director_Baltasar Kormákur,Director_Barry Jenkins,Director_Barry Sonnenfeld,Director_Baz Luhrmann,Director_Ben Affleck,Director_Ben Falcone,Director_Ben Stiller,Director_Bennett Miller,Director_Boaz Yakin,Director_Bobby Farrelly,Director_Bong Joon Ho,Director_Brad Bird,Director_Brad Furman,Director_Brad Peyton,Director_Breck Eisner,...,Director_Richard LaGravenese,Director_Richard Linklater,Director_Rick Famuyiwa,Director_Ridley Scott,Director_Rob Cohen,Director_Rob Letterman,Director_Rob Marshall,Director_Robert Schwentke,Director_Robert Stromberg,Director_Robert Zemeckis,Director_Roland Emmerich,Director_Roman Polanski,Director_Ron Clements,Director_Ron Howard,Director_Ross Katz,Director_Ruben Fleischer,Director_Rupert Sanders,Director_Rupert Wyatt,Director_Ryan Coogler,Director_Sam Mendes,Director_Sam Raimi,Director_Sam Taylor-Johnson,Director_Scott Cooper,Director_Scott Derrickson,Director_Scott Frank,Director_Scott Hicks,Director_Scott Stewart,Director_Scott Waugh,Director_Sean Anders,Director_Sean Penn,Director_Seth Gordon,Director_Seth MacFarlane,Director_Shane Black,Director_Shawn Levy,Director_Simon Curtis,Director_Simon Verhoeven,Director_Simon West,Director_Stephen Chbosky,Director_Stephen Frears,Director_Stephen Gaghan,Director_Steve McQueen,Director_Steven Quale,Director_Steven Soderbergh,Director_Steven Spielberg,Director_Sylvester Stallone,Director_Taika Waititi,Director_Tarsem Singh,Director_Tate Taylor,Director_Thea Sharrock,Director_Thor Freudenthal,Director_Tim Burton,Director_Tim Johnson,Director_Tim Miller,Director_Timur Bekmambetov,Director_Todd Haynes,Director_Todd Phillips,Director_Tom Ford,Director_Tom Gormican,Director_Tom Hooper,Director_Tom McGrath,Director_Tom Tykwer,Director_Tomas Alfredson,Director_Tony Gilroy,Director_Tony Scott,Director_Walt Dohrn,Director_Wes Anderson,Director_Wes Ball,Director_Wes Craven,Director_Whit Stillman,Director_Will Gluck,Director_William Brent Bell,Director_Woody Allen,Director_Yimou Zhang,Director_Yorgos Lanthimos,Director_Zack Snyder
0,0.00,0.123360,0,118.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.45,8.111264,0,103.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [10]:
categorias_encoded_df

,Script Type_adaptation,"Script Type_adaptation, based on a true story","Script Type_adaptation, remake / reboot",Script Type_based on a true story,Script Type_original screenplay,Script Type_remake / reboot,Script Type_sequel,"Script Type_sequel, adaptation",Director_Adam McKay,Director_Adam Shankman,Director_Adam Wingard,Director_Akiva Schaffer,Director_Alan Taylor,Director_Alejandro González Iñárritu,Director_Alessandro Carloni,Director_Alex Garland,Director_Alex Proyas,Director_Alexander Payne,Director_Alfonso Cuarón,Director_Alfonso Gomez-Rejon,Director_Allen Coulter,Director_Amma Asante,Director_Andrew Niccol,Director_Andrew Stanton,Director_Andrés Muschietti,Director_Andy Tennant,Director_Ang Lee,Director_Angelina Jolie,Director_Anna Foerster,Director_Anthony Russo,Director_Antoine Fuqua,Director_Ari Sandel,Director_Ariel Vromen,Director_Asger Leth,Director_Ash Brannon,Director_Babak Najafi,Director_Baltasar Kormákur,Director_Barry Jenkins,Director_Barry Sonnenfeld,Director_Baz Luhrmann,Director_Ben Affleck,Director_Ben Falcone,Director_Ben Stiller,Director_Bennett Miller,Director_Boaz Yakin,Director_Bobby Farrelly,Director_Bong Joon Ho,Director_Brad Bird,Director_Brad Furman,Director_Brad Peyton,Director_Breck Eisner,Director_Bruce A. Evans,Director_Bryan Bertino,Director_Bryan Singer,Director_Byron Howard,Director_Carlos Saldanha,Director_Cary Joji Fukunaga,Director_Catherine Hardwicke,Director_Chad Stahelski,Director_Chris Buck,Director_Chris Columbus,Director_Chris Renaud,Director_Chris Wedge,Director_Chris Weitz,Director_Christophe Lourdelet,Director_Christopher McQuarrie,Director_Christopher Nolan,Director_Clint Eastwood,Director_Colin Trevorrow,Director_Craig Brewer,Director_Craig Gillespie,Director_D.J. Caruso,Director_Damien Chazelle,Director_Dan Gilroy,Director_Dan Mazer,...,Director_Richard LaGravenese,Director_Richard Linklater,Director_Rick Famuyiwa,Director_Ridley Scott,Director_Rob Cohen,Director_Rob Letterman,Director_Rob Marshall,Director_Robert Schwentke,Director_Robert Stromberg,Director_Robert Zemeckis,Director_Roland Emmerich,Director_Roman Polanski,Director_Ron Clements,Director_Ron Howard,Director_Ross Katz,Director_Ruben Fleischer,Director_Rupert Sanders,Director_Rupert Wyatt,Director_Ryan Coogler,Director_Sam Mendes,Director_Sam Raimi,Director_Sam Taylor-Johnson,Director_Scott Cooper,Director_Scott Derrickson,Director_Scott Frank,Director_Scott Hicks,Director_Scott Stewart,Director_Scott Waugh,Director_Sean Anders,Director_Sean Penn,Director_Seth Gordon,Director_Seth MacFarlane,Director_Shane Black,Director_Shawn Levy,Director_Simon Curtis,Director_Simon Verhoeven,Director_Simon West,Director_Stephen Chbosky,Director_Stephen Frears,Director_Stephen Gaghan,Director_Steve McQueen,Director_Steven Quale,Director_Steven Soderbergh,Director_Steven Spielberg,Director_Sylvester Stallone,Director_Taika Waititi,Director_Tarsem Singh,Director_Tate Taylor,Director_Thea Sharrock,Director_Thor Freudenthal,Director_Tim Burton,Director_Tim Johnson,Director_Tim Miller,Director_Timur Bekmambetov,Director_Todd Haynes,Director_Todd Phillips,Director_Tom Ford,Director_Tom Gormican,Director_Tom Hooper,Director_Tom McGrath,Director_Tom Tykwer,Director_Tomas Alfredson,Director_Tony Gilroy,Director_Tony Scott,Director_Walt Dohrn,Director_Wes Anderson,Director_Wes Ball,Director_Wes Craven,Director_Whit Stillman,Director_Will Gluck,Director_William Brent Bell,Director_Woody Allen,Director_Yimou Zhang,Director_Yorgos Lanthimos,Director_Zack Snyder
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

### Normalização dos dados

In [8]:
data_with_target.shape

(506, 380)

In [9]:
data_with_target.to_csv('dados_para_split_revenue.csv',index=False)

In [10]:
# Especifique o nome do arquivo onde você deseja salvar o encoder
nome_arquivo = "one_hot_encoder_com_diretor_revenue.pkl"

# Use a função joblib.dump para salvar o encoder
joblib.dump(encoder, nome_arquivo)

['one_hot_encoder_com_diretor_revenue.pkl']

In [ ]:
# Extrair a coluna que será movida para a última posição
coluna_a_mover = data_with_target['Opening weekend ($million)']

# Remover a coluna do DataFrame original
data_with_target = data_with_target.drop('Opening weekend ($million)', axis=1)

# Adicionar a coluna extraída como última coluna
data_with_target['Opening weekend ($million)'] = coluna_a_mover

In [ ]:
unscaled_inputs = data_with_target.iloc[:,:-1]
unscaled_inputs.head()

,Budget ($million),Oscar Winners,Runtime (Minutes),Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,"Script Type_adaptation, based on a true story","Script Type_adaptation, remake / reboot",Script Type_based on a true story,Script Type_based on a true story,Script Type_original screenplay,Script Type_original screenplay,Script Type_remake / reboot,Script Type_sequel,"Script Type_sequel, adaptation",Director_Adam Shankman,Director_Adam Wingard,Director_Akiva Schaffer,Director_Alan Taylor,Director_Alejandro González Iñárritu,Director_Alessandro Carloni,Director_Alex Garland,Director_Alex Proyas,Director_Alexander Payne,Director_Alfonso Cuarón,Director_Alfonso Gomez-Rejon,Director_Allen Coulter,Director_Amma Asante,Director_Andrew Niccol,Director_Andrew Stanton,Director_Andrés Muschietti,Director_Andy Tennant,Director_Ang Lee,Director_Angelina Jolie,Director_Anna Foerster,Director_Anthony Russo,Director_Antoine Fuqua,Director_Ari Sandel,Director_Ariel Vromen,Director_Asger Leth,Director_Ash Brannon,Director_Babak Najafi,Director_Baltasar Kormákur,Director_Barry Jenkins,Director_Barry Sonnenfeld,Director_Baz Luhrmann,Director_Ben Affleck,Director_Ben Falcone,Director_Ben Stiller,Director_Bennett Miller,Director_Boaz Yakin,Director_Bobby Farrelly,Director_Bong Joon Ho,Director_Brad Bird,Director_Brad Furman,Director_Brad Peyton,Director_Breck Eisner,Director_Bruce A. Evans,...,Director_Richard LaGravenese,Director_Richard Linklater,Director_Rick Famuyiwa,Director_Ridley Scott,Director_Rob Cohen,Director_Rob Letterman,Director_Rob Marshall,Director_Robert Schwentke,Director_Robert Stromberg,Director_Robert Zemeckis,Director_Roland Emmerich,Director_Roman Polanski,Director_Ron Clements,Director_Ron Howard,Director_Ross Katz,Director_Ruben Fleischer,Director_Rupert Sanders,Director_Rupert Wyatt,Director_Ryan Coogler,Director_Sam Mendes,Director_Sam Raimi,Director_Sam Taylor-Johnson,Director_Scott Cooper,Director_Scott Derrickson,Director_Scott Frank,Director_Scott Hicks,Director_Scott Stewart,Director_Scott Waugh,Director_Sean Anders,Director_Sean Penn,Director_Seth Gordon,Director_Seth MacFarlane,Director_Shane Black,Director_Shawn Levy,Director_Simon Curtis,Director_Simon Verhoeven,Director_Simon West,Director_Stephen Chbosky,Director_Stephen Frears,Director_Stephen Gaghan,Director_Steve McQueen,Director_Steven Quale,Director_Steven Soderbergh,Director_Steven Spielberg,Director_Sylvester Stallone,Director_Taika Waititi,Director_Tarsem Singh,Director_Tate Taylor,Director_Thea Sharrock,Director_Thor Freudenthal,Director_Tim Burton,Director_Tim Johnson,Director_Tim Miller,Director_Timur Bekmambetov,Director_Todd Haynes,Director_Todd Phillips,Director_Tom Ford,Director_Tom Gormican,Director_Tom Hooper,Director_Tom McGrath,Director_Tom Tykwer,Director_Tomas Alfredson,Director_Tony Gilroy,Director_Tony Scott,Director_Walt Dohrn,Director_Wes Anderson,Director_Wes Ball,Director_Wes Craven,Director_Whit Stillman,Director_Will Gluck,Director_William Brent Bell,Director_Woody Allen,Director_Yimou Zhang,Director_Yorgos Lanthimos,Director_Zack Snyder
0,-,0,118,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.45,0,103,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.5,0,103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,

In [ ]:
unscaled_inputs.columns

Index(['Budget ($million)', 'Oscar Winners', 'Runtime (Minutes)', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama',
       ...
       'Director_Wes Anderson', 'Director_Wes Ball', 'Director_Wes Craven',
       'Director_Whit Stillman', 'Director_Will Gluck',
       'Director_William Brent Bell', 'Director_Woody Allen',
       'Director_Yimou Zhang', 'Director_Yorgos Lanthimos',
       'Director_Zack Snyder'],
      dtype='object', length=379)

In [ ]:
# from sklearn.preprocessing import StandardScaler

# class CustomScaler(StandardScaler):
#     def __init__(self, columns=None, copy=True, with_mean=True, with_std=True):
#         super().__init__(copy=copy, with_mean=with_mean, with_std=with_std)
#         self.columns = columns
#         self.mean_ = None
#         self.var_ = None

#     def fit(self, X, y=None):
#         if self.columns is not None:
#             self.mean_ = X.loc[:, self.columns].mean()
#             self.var_ = X.loc[:, self.columns].var()
#         else:
#             self.mean_ = X.mean()
#             self.var_ = X.var()
#         return self

#     def transform(self, X, y=None, copy=None):
#         X_copy = X.copy()
#         if self.columns is not None:
#             for col in self.columns:
#                 X_copy[col] = (X_copy[col] - self.mean_[col]) / np.sqrt(self.var_[col])
#         else:
#             X_copy = (X_copy - self.mean_) / np.sqrt(self.var_)
#         return X_copy


In [ ]:
#we must choose the columns to be scaled to see what columns we've got.
unscaled_inputs.columns.values

array(['Budget ($million)', 'Oscar Winners', 'Runtime (Minutes)',
       'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller',
       'War', 'Western', 'Script Type_adaptation, based on a true story',
       'Script Type_adaptation, remake / reboot',
       'Script Type_based on a true story',
       'Script Type_based on a true story ',
       'Script Type_original screenplay',
       'Script Type_original screenplay ', 'Script Type_remake / reboot',
       'Script Type_sequel', 'Script Type_sequel, adaptation',
       'Director_Adam Shankman', 'Director_Adam Wingard',
       'Director_Akiva Schaffer', 'Director_Alan Taylor',
       'Director_Alejandro González Iñárritu',
       'Director_Alessandro Carloni', 'Director_Alex Garland',
       'Director_Alex Proyas', 'Director_Alexander Payne',
       'Director_Alfonso Cuarón', 'Director_

In [ ]:
# columns_to_scale = ['Rotten Tomatoes % critics', 'Metacritic % critics',
#        'Rotten Tomatoes Audience %', 'Metacritic Audience %',
#        'Rotten Tomatoes vs Metacritic % deviance', 'Average audience %',
#        'Audience vs Critics deviance %', 'Opening weekend ($million)', 'Opening Weekend',
#        'Domestic gross ($million)', 'Domestic Gross',
#        'Foreign Gross ($million)', 'Foreign Gross', 'Worldwide Gross',
#        'Worldwide Gross ($million)', '% of Gross earned abroad',
#        'Budget ($million)', '% Budget recovered opening weekend',
#        'Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)',
#        'Metascore']

In [ ]:
# #Let's created an empty standard scalar object
# film_scaler = StandardScaler()
# film_scaler = CustomScaler(columns_to_scale)

In [ ]:
# #fit our input data, this will calculate the mean and standard deviation of each feature from unskilled inputs
# film_scaler.fit(unscaled_inputs)

CustomScaler(columns=['Rotten Tomatoes % critics', 'Metacritic % critics',
                      'Rotten Tomatoes Audience %', 'Metacritic Audience %',
                      'Rotten Tomatoes vs Metacritic % deviance',
                      'Average audience %', 'Audience vs Critics deviance %',
                      'Opening weekend ($million)', 'Opening Weekend',
                      'Domestic gross ($million)', 'Domestic Gross',
                      'Foreign Gross ($million)', 'Foreign Gross',
                      'Worldwide Gross', 'Worldwide Gross ($million)',
                      '% of Gross earned abroad', 'Budget ($million)',
                      '% Budget recovered opening weekend', 'Runtime (Minutes)',
                      'Rating', 'Votes', 'Revenue (Millions)', 'Metascore'])

In [ ]:
# #Until we transform the data inputs still on non-scaled. For using this method we always transform new data
# scaled_inputs = film_scaler.transform(unscaled_inputs)

In [ ]:
# # you can see all our dummies have remained untouched and that was the whole point
# scaled_inputs.head()

In [ ]:
# scaled_inputs.shape

### Salvar features e target

In [ ]:
# X_features = unscaled_inputs.copy()
# y_target = targets.copy()
# df = data_with_target.copy()

In [ ]:
# X_features.to_excel('X_features.xlsx',index=False)
# np.savetxt("y_target.csv", y_target, delimiter=",")
# df.to_excel('df_with_target.xlsx',index=False)